In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from collections import Counter, defaultdict
import random
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

class IALabo:
    """Système d'analyse intelligent pour jeux de hasard"""
    
    def __init__(self, fichier_donnees):
        """
        Initialise IA Labo
        :param fichier_donnees: Chemin vers le fichier historique
        """
        # Définir les attributs d'abord
        self.numero_max = 49  # Pour Loto français
        self.historique_scores = defaultdict(list)
        self.performance_reseaux = {}
        
        # Puis charger les données
        self.df = self.charger_donnees(fichier_donnees)
        
    def charger_donnees(self, fichier):
        """Charge et prépare les données historiques"""
        try:
            print(f"📖 Chargement du fichier: {fichier}")
            
            # Charger avec séparateur point-virgule
            df = pd.read_csv("lotodata.csv", sep=';')
            
            print(f"✅ {len(df)} lignes chargées")
            print(f"Colonnes: {df.columns.tolist()}")
            print(f"Aperçu:\n{df.head()}")
            
            # Vérifier que nous avons les bonnes colonnes
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y', errors='coerce')
            
            # Vérifier les colonnes de numéros
            numeros_cols = []
            for i in range(5):
                col_name = f'num{i}'
                if col_name in df.columns:
                    df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
                    numeros_cols.append(col_name)
            
            # S'il manque des colonnes
            if len(numeros_cols) < 5:
                print(f"⚠️  Colonnes numériques disponibles: {numeros_cols}")
                # Chercher d'autres colonnes numériques
                numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
                for col in numeric_cols:
                    if col not in numeros_cols and len(numeros_cols) < 5:
                        new_name = f'num{len(numeros_cols)}'
                        df[new_name] = df[col]
                        numeros_cols.append(new_name)
            
            # Garder seulement les colonnes de numéros
            df = df[numeros_cols[:5]].copy()
            
            # Nettoyage
            df = df.dropna()
            for col in df.columns:
                df[col] = df[col].astype(int)
            
            # Filtrer les valeurs valides
            for col in df.columns:
                df = df[(df[col] >= 1) & (df[col] <= self.numero_max)]
            
            # Trier les numéros
            df = pd.DataFrame(np.sort(df.values, axis=1), columns=df.columns)
            
            print(f"✅ {len(df)} tirages valides après nettoyage")
            return df
            
        except Exception as e:
            print(f"❌ Erreur lors du chargement: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    # ==================== MODULES AVANCÉS ====================
    
    class ReseauNeurones:
        """Module 1: Vrai réseau de neurones pour patterns complexes"""
        
        def __init__(self, historique):
            self.historique = historique
            self.model = None
            self.scaler = StandardScaler()
            self.entrainer_reseau()
            
        def preparer_donnees(self):
            """Prépare les données pour l'entraînement"""
            if len(self.historique) < 50:
                return None, None
            
            # Créer des features à partir de l'historique
            X = []
            y = []
            
            for i in range(len(self.historique) - 1):
                tirage_actuel = self.historique[i]
                tirage_suivant = self.historique[i+1]
                
                # Features: fréquences, retards, patterns
                features = self.extraire_features(tirage_actuel, i)
                X.append(features)
                
                # Target: apparition dans le prochain tirage
                target = np.zeros(49)
                for num in tirage_suivant:
                    target[num-1] = 1
                y.append(target)
            
            return np.array(X), np.array(y)
        
        def extraire_features(self, tirage, index):
            """Extrait des features d'un tirage"""
            features = []
            
            # 1. Fréquences des numéros dans les 50 derniers tirages
            fenetre = self.historique[max(0, index-50):index]
            freq = Counter()
            for t in fenetre:
                freq.update(t)
            
            for num in range(1, 50):
                features.append(freq.get(num, 0) / len(fenetre) if fenetre else 0)
            
            # 2. Délai depuis la dernière apparition
            for num in range(1, 50):
                delai = 0
                for j in range(index-1, max(0, index-100)-1, -1):
                    if num in self.historique[j]:
                        break
                    delai += 1
                features.append(delai / 100)
            
            # 3. Caractéristiques du tirage actuel
            features.append(sum(tirage) / 250)  # Somme normalisée
            features.append(sum(1 for n in tirage if n % 2 == 0) / 5)  # Ratio pairs
            features.append(len(set((n-1)//10 for n in tirage)) / 5)  # Diversité dizaines
            
            return features
        
        def entrainer_reseau(self):
            """Entraîne le réseau de neurones"""
            X, y = self.preparer_donnees()
            if X is None or len(X) < 30:
                print("   ⚠️  Pas assez de données pour le réseau de neurones")
                return
            
            # Normaliser les features
            X_scaled = self.scaler.fit_transform(X)
            
            # Créer et entraîner le modèle
            self.model = MLPRegressor(
                hidden_layer_sizes=(100, 50, 25),
                activation='relu',
                solver='adam',
                max_iter=1000,
                random_state=42
            )
            
            self.model.fit(X_scaled, y)
            print("   ✅ Réseau de neurones entraîné")
        
        def analyser(self):
            """Prédit les probabilités pour le prochain tirage"""
            scores = np.zeros(49)
            
            if self.model is None or len(self.historique) < 10:
                # Fallback: fréquence simple
                for tirage in self.historique[-20:]:
                    for num in tirage:
                        scores[num-1] += 1
                if scores.sum() > 0:
                    scores = scores / scores.sum()
                return scores
            
            # Utiliser le dernier tirage pour prédire
            dernier_tirage = self.historique[-1]
            features = self.extraire_features(dernier_tirage, len(self.historique)-1)
            features_scaled = self.scaler.transform([features])
            
            # Prédiction
            predictions = self.model.predict(features_scaled)[0]
            scores = predictions
            
            return scores
    
    class AlgorithmeGenetique:
        """Module 2: Algorithme génétique pour exploration adaptative"""
        
        def __init__(self, historique):
            self.historique = historique
            self.population_size = 100
            self.generations = 100
            self.mutation_rate = 0.1
            
        def analyser(self):
            """Recherche de combinaisons optimales par évolution"""
            scores = np.zeros(49)
            
            # Initialiser la population
            population = self.initialiser_population()
            
            # Évolution
            for generation in range(self.generations):
                # Évaluation
                fitness = self.evaluer_population(population)
                
                # Sélection
                parents = self.selectionner_parents(population, fitness)
                
                # Reproduction
                population = self.reproduire(parents)
            
            # Compter les apparitions dans la population finale
            for individu in population:
                for num in individu:
                    scores[num-1] += 1
            
            return scores / scores.sum() if scores.sum() > 0 else scores
        
        def initialiser_population(self):
            """Initialise une population aléatoire"""
            population = []
            for _ in range(self.population_size):
                individu = random.sample(range(1, 50), 5)
                population.append(sorted(individu))
            return population
        
        def evaluer_population(self, population):
            """Évalue la fitness de chaque individu"""
            fitness = []
            for individu in population:
                score = 0
                
                # 1. Diversité (éviter les répétitions)
                score += len(set(individu)) * 2
                
                # 2. Distance par rapport à l'historique récent
                for tirage in self.historique[-20:]:
                    communs = len(set(individu) & set(tirage))
                    score -= communs * 0.5  # Pénalise la similarité
                
                # 3. Équilibre pairs/impairs
                pairs = sum(1 for n in individu if n % 2 == 0)
                if 1 <= pairs <= 4:
                    score += 2
                
                fitness.append(score)
            
            return np.array(fitness)
        
        def selectionner_parents(self, population, fitness):
            """Sélectionne les parents par tournoi"""
            parents = []
            for _ in range(self.population_size):
                # Tournoi de taille 3
                candidates = random.sample(range(len(population)), 3)
                best = max(candidates, key=lambda i: fitness[i])
                parents.append(population[best])
            return parents
        
        def reproduire(self, parents):
            """Crée une nouvelle génération"""
            new_population = []
            
            for i in range(0, len(parents), 2):
                if i+1 < len(parents):
                    parent1, parent2 = parents[i], parents[i+1]
                    
                    # Croisement
                    enfant = self.croisement(parent1, parent2)
                    
                    # Mutation
                    enfant = self.muter(enfant)
                    
                    new_population.append(enfant)
            
            return new_population
        
        def croisement(self, parent1, parent2):
            """Croisement à un point"""
            point = random.randint(1, 3)
            enfant = parent1[:point] + parent2[point:]
            enfant = sorted(list(set(enfant)))  # Éliminer les doublons
            
            # Compléter si nécessaire
            while len(enfant) < 5:
                nouveau = random.randint(1, 49)
                if nouveau not in enfant:
                    enfant.append(nouveau)
            
            return sorted(enfant)
        
        def muter(self, individu):
            """Mutation aléatoire"""
            if random.random() < self.mutation_rate:
                idx = random.randint(0, 4)
                nouveau = random.randint(1, 49)
                while nouveau in individu:
                    nouveau = random.randint(1, 49)
                individu[idx] = nouveau
            
            return sorted(individu)
    
    class ChainesMarkov:
        """Module 3: Chaînes de Markov pour dépendances temporelles"""
        
        def __init__(self, historique):
            self.historique = historique
            self.matrice_transition = self.calculer_matrice()
            
        def calculer_matrice(self):
            """Calcule la matrice de transition entre numéros"""
            matrice = np.zeros((49, 49))
            
            for i in range(len(self.historique) - 1):
                tirage_actuel = self.historique[i]
                tirage_suivant = self.historique[i+1]
                
                for num1 in tirage_actuel:
                    for num2 in tirage_suivant:
                        matrice[num1-1, num2-1] += 1
            
            # Normaliser
            somme_lignes = matrice.sum(axis=1)
            somme_lignes[somme_lignes == 0] = 1
            matrice = matrice / somme_lignes[:, np.newaxis]
            
            return matrice
        
        def analyser(self):
            """Prédit les probabilités"""
            if len(self.historique) == 0:
                return np.zeros(49)
            
            derniers_numeros = self.historique[-1]
            scores = np.zeros(49)
            
            for num in derniers_numeros:
                scores += self.matrice_transition[num-1]
            
            return scores / len(derniers_numeros)
    
    class AnalyseStatistique:
        """Module 4: Analyse statistique avancée"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse multi-critères"""
            scores = np.zeros(49)
            
            # 1. Fréquences pondérées (récent > ancien)
            poids = np.linspace(0.1, 1.0, min(50, len(self.historique)))
            for i, tirage in enumerate(self.historique[-len(poids):]):
                for num in tirage:
                    scores[num-1] += poids[i]
            
            # 2. Retards exponentiels
            delais = self.calculer_delais()
            max_delai = max(delais.values()) if delais else 1
            for num, delai in delais.items():
                scores[num-1] += (delai / max_delai) * 0.5
            
            # 3. Équilibre statistique
            scores = self.appliquer_equilibre(scores)
            
            return scores / scores.sum() if scores.sum() > 0 else scores
        
        def calculer_delais(self):
            """Calcule les délais depuis la dernière apparition"""
            delais = {}
            for num in range(1, 50):
                delai = 0
                for tirage in reversed(self.historique):
                    delai += 1
                    if num in tirage:
                        break
                delais[num] = delai
            return delais
        
        def appliquer_equilibre(self, scores):
            """Applique des critères d'équilibre"""
            scores_adj = scores.copy()
            
            # Réduire les extrêmes
            mean_score = scores.mean()
            std_score = scores.std()
            scores_adj = np.clip(scores, mean_score - std_score, mean_score + std_score)
            
            return scores_adj
    
    class AnalyseFrequentielle:
        """Module 5: Analyse fréquentielle classique"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse des fréquences"""
            scores = np.zeros(49)
            
            if not self.historique:
                return scores
            
            # Fréquences globales
            compteur = Counter()
            for tirage in self.historique:
                compteur.update(tirage)
            
            total = sum(compteur.values())
            if total > 0:
                for num, count in compteur.items():
                    scores[num-1] = count / total
            
            return scores
    
    class AnalyseGeometrique:
        """Module 6: Analyse géométrique"""
        
        def __init__(self, historique):
            self.historique = historique
            
        def analyser(self):
            """Analyse des patterns géométriques"""
            scores = np.zeros(49)
            
            # 1. Clusters géométriques
            clusters = self.detecter_clusters()
            for cluster in clusters:
                for num in cluster:
                    scores[num-1] += 0.1
            
            # 2. Répartition spatiale
            repartition = self.analyser_repartition()
            for num, score in repartition.items():
                scores[num-1] += score * 0.2
            
            # 3. Patterns de grille
            patterns = self.detecter_patterns()
            for pattern in patterns:
                for num in pattern:
                    scores[num-1] += 0.05
            
            return scores / scores.sum() if scores.sum() > 0 else scores
        
        def detecter_clusters(self):
            """Détecte les clusters de numéros proches"""
            clusters = []
            # Implémentation simplifiée
            for tirage in self.historique[-50:]:
                if len(tirage) >= 3:
                    # Chercher des numéros à distance ≤ 10
                    for i in range(len(tirage)):
                        cluster = [tirage[i]]
                        for j in range(i+1, len(tirage)):
                            if abs(tirage[j] - tirage[i]) <= 10:
                                cluster.append(tirage[j])
                        if len(cluster) >= 3:
                            clusters.append(cluster)
            return clusters
        
        def analyser_repartition(self):
            """Analyse la répartition spatiale sur grille 7x7"""
            repartition = {}
            for num in range(1, 50):
                ligne = (num - 1) // 7
                colonne = (num - 1) % 7
                # Score basé sur la position (centre > bords)
                distance_centre = abs(ligne - 3) + abs(colonne - 3)
                repartition[num] = 1 - (distance_centre / 6)
            return repartition
        
        def detecter_patterns(self):
            """Détecte des patterns visuels"""
            patterns = []
            # À implémenter
            return patterns
    
    # ==================== MOTEUR D'AGRÉGATION ====================
    
    def analyser_avec_tous_modules(self):
        """Exécute tous les modules d'analyse"""
        if self.df is None or len(self.df) < 30:
            print("❌ Pas assez de données historiques (min 30 tirages)")
            return None
        
        # Préparer les données historiques
        numeros_columns = [f'num{i}' for i in range(5)]
        historique_tirages = self.df[numeros_columns].values.tolist()
        
        print(f"📊 Analyse de {len(historique_tirages)} tirages avec 6 modules...")
        
        # Initialiser les modules
        modules = {
            'Reseau Neurones': self.ReseauNeurones(historique_tirages),
            'Algo Génétique': self.AlgorithmeGenetique(historique_tirages),
            'Chaînes Markov': self.ChainesMarkov(historique_tirages),
            'Statistique': self.AnalyseStatistique(historique_tirages),
            'Fréquentiel': self.AnalyseFrequentielle(historique_tirages),
            'Géométrique': self.AnalyseGeometrique(historique_tirages)
        }
        
        # Exécuter toutes les analyses
        resultats = {}
        for nom, module in modules.items():
            print(f"   🧠 Exécution: {nom}...")
            scores = module.analyser()
            resultats[nom] = scores
        
        print("✅ Tous les modules terminés")
        return resultats
    
    def bagging(self, resultats_modules):
        """Méthode Bagging: moyenne simple"""
        if not resultats_modules:
            return None
        
        scores_combined = np.zeros(self.numero_max)
        
        for module_name, scores in resultats_modules.items():
            scores_combined += scores
        
        scores_combined /= len(resultats_modules)
        
        # Classement
        classement = np.argsort(scores_combined)[::-1] + 1
        
        return {
            'scores': scores_combined,
            'classement': classement[:20],
            'methode': 'bagging'
        }
    
    def boosting(self, resultats_modules):
        """Méthode Boosting: pondération intelligente"""
        if not resultats_modules:
            return None
        
        # Pondérations basées sur la complexité des modèles
        poids = {
            'Reseau Neurones': 1.5,  # Modèle complexe
            'Algo Génétique': 1.3,    # Exploration adaptative
            'Chaînes Markov': 1.2,    # Dépendances temporelles
            'Statistique': 1.0,       # Analyse classique
            'Fréquentiel': 0.8,       # Simple mais efficace
            'Géométrique': 0.9        # Patterns visuels
        }
        
        scores_combined = np.zeros(self.numero_max)
        poids_total = sum(poids.values())
        
        for module_name, scores in resultats_modules.items():
            poids_module = poids.get(module_name, 1.0)
            scores_combined += scores * poids_module
        
        scores_combined /= poids_total
        
        # Classement
        classement = np.argsort(scores_combined)[::-1] + 1
        
        return {
            'scores': scores_combined,
            'classement': classement[:20],
            'methode': 'boosting',
            'poids': poids
        }
    
    def generer_recommandations(self, methode='bagging'):
        """Génère des recommandations finales"""
        print(f"\n{'='*60}")
        print(f"🧠 IA LABO - Génération de recommandations")
        print(f"{'='*60}")
        
        # 1. Analyser avec tous les modules
        resultats_modules = self.analyser_avec_tous_modules()
        if resultats_modules is None:
            return
        
        # 2. Appliquer l'agrégation
        if methode == 'bagging':
            resultat = self.bagging(resultats_modules)
        else:
            resultat = self.boosting(resultats_modules)
        
        if resultat is None:
            return
        
        # 3. Afficher les résultats
        self.afficher_resultats(resultat, resultats_modules)
        
        # 4. Générer des combinaisons
        if len(resultat['classement']) >= 12:
            combinaisons = self.generer_combinaisons_avancees(resultat['classement'][:15])
            self.afficher_combinaisons(combinaisons)
        
        return resultat
    
    def afficher_resultats(self, resultat_aggrege, resultats_modules):
        """Affiche les résultats d'analyse"""
        print(f"\n📊 RÉSULTATS - {resultat_aggrege['methode'].upper()}")
        
        if 'poids' in resultat_aggrege:
            print("\n⚖️  PONDÉRATIONS DES MODÈLES:")
            for module, poids in sorted(resultat_aggrege['poids'].items(), key=lambda x: x[1], reverse=True):
                print(f"   {module:20} → {poids:.2f}")
        
        print(f"\n🏆 TOP 15 NUMÉROS (Score relatif):")
        print("   Rang | Numéro | Score  | Type      | Dizaine")
        print("   " + "-" * 50)
        
        for i, num in enumerate(resultat_aggrege['classement'][:15], 1):
            score = resultat_aggrege['scores'][num-1]
            type_num = "Pair" if num % 2 == 0 else "Impair"
            dizaine = f"{(num-1)//10 * 10 + 1:02d}-{(num-1)//10 * 10 + 10:02d}"
            
            print(f"   {i:4} | {num:6} | {score:.4f} | {type_num:8} | {dizaine}")
    
    def generer_combinaisons_avancees(self, numeros_recommandes, n_combinaisons=5):
        """Génère des combinaisons avec critères avancés"""
        combinaisons = []
        
        for _ in range(n_combinaisons * 3):  # Plus d'essais
            if len(combinaisons) >= n_combinaisons:
                break
            
            # Sélection intelligente
            combinaison = self.selectionner_intelligemment(numeros_recommandes)
            
            if self.combinaison_valide_avancee(combinaison):
                combinaisons.append(sorted(combinaison))
        
        return list(set([tuple(c) for c in combinaisons]))[:n_combinaisons]  # Éliminer les doublons
    
    def selectionner_intelligemment(self, numeros_recommandes):
        """Sélectionne des numéros de façon intelligente"""
        # Probabilités basées sur les scores
        scores = np.array([self.calculer_score_combinaison(numeros_recommandes)])
        probas = scores / scores.sum()
        
        # Sélection sans remise
        selected = np.random.choice(
            numeros_recommandes,
            size=5,
            replace=False,
            p=probas[0] if probas[0].sum() > 0 else None
        )
        
        return sorted(selected.tolist())
    
    def calculer_score_combinaison(self, numeros):
        """Calcule un score pour une liste de numéros"""
        scores = np.random.rand(len(numeros)) * 0.5 + 0.5  # Entre 0.5 et 1.0
        return scores
    
    def combinaison_valide_avancee(self, combinaison):
        """Critères avancés de validation"""
        # 1. Équilibre pairs/impairs
        pairs = sum(1 for n in combinaison if n % 2 == 0)
        if not (1 <= pairs <= 4):
            return False
        
        # 2. Diversité des dizaines
        dizaines = set((n-1)//10 for n in combinaison)
        if len(dizaines) < 2:
            return False
        
        # 3. Somme raisonnable
        somme = sum(combinaison)
        if not (100 <= somme <= 200):
            return False
        
        # 4. Écart minimum entre numéros
        for i in range(len(combinaison)-1):
            if combinaison[i+1] - combinaison[i] < 3:  # Trop proches
                return False
        
        # 5. Pas de patterns trop évidents
        if self.pattern_trop_regulier(combinaison):
            return False
        
        return True
    
    def pattern_trop_regulier(self, combinaison):
        """Détecte les patterns trop réguliers"""
        # Vérifier les suites arithmétiques
        differences = [combinaison[i+1] - combinaison[i] for i in range(len(combinaison)-1)]
        if len(set(differences)) == 1:  # Toutes les différences égales
            return True
        
        # Vérifier les multiples
        for i in range(len(combinaison)):
            for j in range(i+1, len(combinaison)):
                if combinaison[j] % combinaison[i] == 0:
                    return True
        
        return False
    
    def afficher_combinaisons(self, combinaisons):
        """Affiche les combinaisons suggérées"""
        print(f"\n🎯 {len(combinaisons)} COMBINAISONS OPTIMISÉES:")
        print("   (Critères: équilibre, diversité, somme 100-200, pas de patterns évidents)")
        
        for i, comb in enumerate(combinaisons, 1):
            somme = sum(comb)
            pairs = sum(1 for n in comb if n % 2 == 0)
            dizaines = len(set((n-1)//10 for n in comb))
            ecart = max(comb) - min(comb)
            
            print(f"\n   Option {i}: {comb}")
            print(f"      • Analyse:")
            print(f"        Somme: {somme:3d} (moyenne: {somme/5:5.1f})")
            print(f"        Pairs/Impairs: {pairs}/{5-pairs}")
            print(f"        Dizaines: {dizaines}/5")
            print(f"        Écart: {ecart:3d}")
            print(f"        Écart moyen: {ecart/4:5.1f}")

# ==================== UTILISATION ====================

if __name__ == "__main__":
    print("""
    ██╗ █████╗      ██╗      █████╗ ██████╗  ██████╗ 
    ██║██╔══██╗     ██║     ██╔══██╗██╔══██╗██╔═══██╗
    ██║███████║     ██║     ███████║██████╔╝██║   ██║
    ██║██╔══██║██   ██║     ██╔══██║██╔══██╗██║   ██║
    ██║██║  ██║╚█████╔╝     ██║  ██║██████╔╝╚██████╔╝
    ╚═╝╚═╝  ╚═╝ ╚════╝      ╚═╝  ╚═╝╚═════╝  ╚═════╝ 
    
    IA Labo - Système d'analyse avancé pour Loto
    Version 2.0 avec Réseau de Neurones et Algorithme Génétique
    """)
    
    try:
        # Initialiser
        print("\n🚀 INITIALISATION DU SYSTÈME IA...")
        ia_labo = IALabo('lotodata1.csv')
        
        if ia_labo.df is not None and len(ia_labo.df) >= 30:
            print(f"\n✅ {len(ia_labo.df)} tirages prêts pour l'analyse")
            
            # Bagging
            print("\n" + "="*60)
            print("1. MÉTHODE BAGGING (MOYENNE DES 6 MODÈLES)")
            print("="*60)
            resultats_bagging = ia_labo.generer_recommandations(methode='bagging')
            
            # Boosting
            print("\n" + "="*60)
            print("2. MÉTHODE BOOSTING (PONDÉRATION INTELLIGENTE)")
            print("="*60)
            resultats_boosting = ia_labo.generer_recommandations(methode='boosting')
            
            print("\n" + "="*60)
            print("📋 RAPPEL:")
            print("""
            • IA Labo combine 6 méthodes d'analyse avancées
            • L'agrégation réduit les biais individuels
            • Les résultats sont statistiquement optimisés
            • Aucune garantie de gain - outil de recherche
            """)
            
        else:
            print(f"\n❌ Données insuffisantes: {len(ia_labo.df) if ia_labo.df is not None else 0} tirages")
            print("   Minimum requis: 30 tirages historiques")
            
    except Exception as e:
        print(f"\n❌ ERREUR: {e}")
        import traceback
        traceback.print_exc()


    ██╗ █████╗      ██╗      █████╗ ██████╗  ██████╗ 
    ██║██╔══██╗     ██║     ██╔══██╗██╔══██╗██╔═══██╗
    ██║███████║     ██║     ███████║██████╔╝██║   ██║
    ██║██╔══██║██   ██║     ██╔══██║██╔══██╗██║   ██║
    ██║██║  ██║╚█████╔╝     ██║  ██║██████╔╝╚██████╔╝
    ╚═╝╚═╝  ╚═╝ ╚════╝      ╚═╝  ╚═╝╚═════╝  ╚═════╝ 
    
    IA Labo - Système d'analyse avancé pour Loto
    Version 2.0 avec Réseau de Neurones et Algorithme Génétique
    

🚀 INITIALISATION DU SYSTÈME IA...
📖 Chargement du fichier: lotodata1.csv
✅ 951 lignes chargées
Colonnes: ['date', 'num0', 'num1', 'num2', 'num3', 'num4']
Aperçu:
         date  num0  num1  num2  num3  num4
0  01/12/2025    38    46    11    13    14
1  29/11/2025     8    13    20    27    23
2  26/11/2025    14    16    26     9    33
3  24/11/2025    47    29    36    28    22
4  22/11/2025     7    10    45    34    40
✅ 951 tirages valides après nettoyage

✅ 951 tirages prêts pour l'analyse

1. MÉTHODE BAGGING (MOYENNE DES 6 MODÈLES)

🧠 IA L